In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [2]:
#chrome driver가 설치될 파일 경로 확인
import os
os.getcwd()

'C:\\Users\\EunSeon\\anaconda3\\envs\\week1'

In [82]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [84]:
#사고년도 설정 (2021년)
year_start=driver.find_element(By.XPATH, '//*[@id="ptsRafYearStart"]/option[3]')
year_start.click()
year_end=driver.find_element(By.XPATH, '//*[@id="ptsRafYearEnd"]/option[3]')
year_end.click()

In [85]:
#지역(제주도) 설정
region=driver.find_element(By.XPATH, '//*[@id="ptsRafSido"]/option[17]')
region.click()

In [86]:
#제주도 지역 전체 설정
select=driver.find_element(By.XPATH, '//*[@id="ptsRafSigungu"]/option[1]')
select.click()

In [87]:
#모든 사고 유형 선택
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[2]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[3]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[4]/input')
acc.click()

In [167]:
#사고부문 선택 (어린이 관련 사고로 한정)
opt=driver.find_element(By.XPATH,'//*[@id="ptsRafSimpleCondition"]/option[16]')
opt.click()

In [168]:
#검색
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)

In [172]:
tn=(driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[3]/div[1]/span').text)
tn=int(tn.replace(",",""))

In [173]:
tn

17

In [174]:
#크롤링 함수 정의 (num 변수는 총 교통사고 수를 의미)
#coord_ls는 좌표계, srs_ls는 좌표계 종류를 의미하는 리스트
def crawling(num):
    coord_ls = []
    srs_ls = []
    for i in tqdm(range(0,num)):
        driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
        time.sleep(1) # 로드
        bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
        src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte
        src_split = src.split('&')
        bbox_text = src_split[-3][5:]
        coords = [float(i) for i in bbox_text.split(',')]
        x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
        srs = src_split[-4][4:]
        coord_ls.append([x, y])
        srs_ls.append(srs)
    return coord_ls, srs_ls

In [175]:
#크롤링 진행
coord,srs=crawling(tn)
coord=np.array(coord)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:20<00:00,  1.21s/it]


In [123]:
#좌표 변환 함수
import pyproj
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5181
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fy, fx])[0]

In [176]:
# 좌표계 정보 설정
p1_type = "epsg:5181"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)

C:\Users\EunSeon\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EunSeon\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\EunSeon\AppData\Local\Temp\ipykernel_58844\3994619643.py:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  fx, fy

In [177]:
df=pd.DataFrame(result,columns=['위도','경도'])

In [99]:
from PyKakao import Local
api = Local(service_key = "카카오API에서받은키넣기")

In [178]:
df['행정동']=None
for i in range(0,len(df)):
    #데이터 = (경도, 위도) -> 행정동으로 변환
    hd=api.geo_coord2regioncode(df.iloc[i,1],df.iloc[i,0],dataframe=True)
    #맨 마지막 열에 변환된 행정동 저장
    df.iloc[i,2]=hd.iloc[1,5]

In [179]:
df.tail()

,위도,경도,행정동
12,33.357266,126.295822,한림읍
13,33.501009,126.548726,일도2동
14,33.533145,126.619475,조천읍
15,33.520433,126.585318,삼양동
16,33.554752,126.750570,구좌읍


In [180]:
final_df=pd.concat([final_df,df],axis=0)

In [181]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 378 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   위도      378 non-null    float64
 1   경도      378 non-null    float64
 2   행정동     378 non-null    object 
dtypes: float64(2), object(1)
memory usage: 11.8+ KB


In [185]:
hd

,region_type,code,address_name,region_1depth_name,region_2depth_name,region_3depth_name,region_4depth_name,x,y
0,B,5011025634,제주특별자치도 제주시 구좌읍 김녕리,제주특별자치도,제주시,구좌읍,김녕리,126.754896,33.539961
1,H,5011025600,제주특별자치도 제주시 구좌읍,제주특별자치도,제주시,구좌읍,,126.852041,33.522552


In [183]:
final_df.to_csv('사고건수.csv',index=False,encoding='cp949')